In [ ]:
import random as r
import math as m
import pulp as p
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

def locate(f,c):
    n_c=len(c)
    n_f=len(f)
    xf=[0]*n_f
    yf=[0]*n_f
    for j in range(n_f):
        xf[j]=lng[j]
        yf[j]=lat[j]
    plt.plot(xf,yf,'v',color='blue')
    
    return [xf,yf]

def showsol(f,c,sol,pnt):
    colors=[]
    for i in range(30):
        colors.append('#%06X'%r.randint(0,0xFFFFFF))
    n_c=len(c)
    n_f=len(f)
    for j in range(n_f):
        if sol[j]:
            #plt.plot(pnt[2][j],pnt[3][j],'v',color=colors[j%(len(colors)-1)],markersize=15)
            plt.plot(pnt[0][j],pnt[1][j],'v',color='red',markersize=15)
        else:
            #plt.plot(pnt[2][j],pnt[3][j],'v',color='blue')
            plt.plot(pnt[0][j],pnt[1][j],'v',color='blue', alpha=0.5, markersize=3)
    for i in range(n_c):
        cf=1000000
        el=-1
        for j in range(n_f):
            if sol[j]:
                if cf>c[i][j]:
                    cf=c[i][j]
                    el=j
        #plt.plot(pnt[0][i],pnt[1][i],'o',color=colors[el%(len(colors)-1)])
        #plt.plot(pnt[0][i],pnt[1][i],'o',color='black', alpha=0.5)
    plt.show()

def ilp_model(f,c,pVal):
    
    n_c=len(c)
    n_f=len(f)
    
    model=p.LpProblem('P-Median',p.LpMinimize)
    
    #we create y as the list of decisional binary variables active if we use the corrisponding facility
    y=[0]*n_f
    for j in range(n_f):
        nameV='y'+str(j)
        y[j]=p.LpVariable(name=nameV,cat=p.LpBinary)
    
    #we create x as the matrix of decisional binary variables
    x=[0]*n_c
    for i in range(n_c):
        x[i]=[0]*n_f
        for j in range(n_f):
            nameV='x'+str(i)+'_'+str(j)
            x[i][j]=p.LpVariable(name=nameV,cat=p.LpBinary)
    
    #objective function
    obj=p.LpAffineExpression()
    
    #fixed cost
    for j in range(n_f):
        obj+=f[j]*y[j]
    
    #transportation cost
    for i in range(n_c):
        for j in range(n_f):
            obj+=c[i][j]*x[i][j]
    model+=obj
    
    #constraints each customer is served by a single facility
    for i in range(n_c):
        c1=p.LpAffineExpression()
        for j in range(n_f):
            c1+=x[i][j]
        model+=c1==1

    for i in range(n_f):
        c2=p.LpAffineExpression()
        for j in range(n_c):
            c2+=x[j][i]*population[i]
        model+=c2<=5000000
    
    #the customer i is served by only an active facility
    for i in range(n_c):
        for j in range(n_f):
            model+=x[i][j]<=y[j]
 
    #set the required number of active facilities
    c3=p.LpAffineExpression()
    for j in range(n_f):
        c3+=y[j]
    model+=c3==pVal
    
    #f=open('model.pl','w')
    #f.write(str(model))
    #f.close()
    
    #solve
    status=model.solve()
    print('Status:',p.LpStatus[status])
    obj=p.value(model.objective)
    print('Objective Value:',obj)
    out=[False]*n_f
    for j in range(n_f):
        if p.value(y[j])>0.5:
            out[j]=True
    return out

def SPL(f,c, population):
    
    n_c=len(c)
    n_f=len(f)
    
    model=p.LpProblem('P-Median',p.LpMinimize)
    
    #we create y as the list of decisional binary variables active if we use the corrisponding facility
    y=[0]*n_f
    for j in range(n_f):
        nameV='y'+str(j)
        y[j]=p.LpVariable(name=nameV,cat=p.LpBinary)
    
    #we create x as the matrix of decisional binary variables
    x=[0]*n_c
    for i in range(n_c):
        x[i]=[0]*n_f
        for j in range(n_f):
            nameV='x'+str(i)+'_'+str(j)
            x[i][j]=p.LpVariable(name=nameV,cat=p.LpBinary)
    
    #objective function
    obj=p.LpAffineExpression()
    
    #fixed cost
    for j in range(n_f):
        obj+=f[j]*y[j]
    
    #transportation cost
    for i in range(n_c):
        for j in range(n_f):
            obj+=c[i][j]*x[i][j]
    model+=obj
    
    #constraints each customer is served by a single facility
    for i in range(n_c):
        c1=p.LpAffineExpression()
        for j in range(n_f):
            c1+=x[i][j]
        model+=c1==1
    
    for i in range(n_f):
        c2=p.LpAffineExpression()
        for j in range(n_c):
            c2+=x[j][i]*population[i]
        model+=c2<=5000000

    #the customer i is served by only an active facility
    for i in range(n_c):
        for j in range(n_f):
            model+=x[i][j]<=y[j]
 
    #f=open('model.pl','w')
    #f.write(str(model))
    #f.close()
    
    #solve
    status=model.solve()
    print('Status:',p.LpStatus[status])
    obj=p.value(model.objective)
    print('Objective Value:',obj)
    out=[False]*n_f
    for j in range(n_f):
        if p.value(y[j])>0.5:
            out[j]=True
    return out

def constructive(f,c,pVal):
    n_c=len(c)
    n_f=len(f)
    out=[False]*n_f
    for k in range(pVal):
        selected=-1
        while selected==-1:
            j=r.randrange(0,n_f)
            if not out[j]:
                selected=j
        if selected>=0:
            out[selected]=True
    return out

def evaluate_out(f,c,ris):
    n_c=len(c)
    n_f=len(f)
    cost=0
    for j in range(n_f):
        if ris[j]:
            cost+=f[j]
    for i in range(n_c):
        cf=1000000
        for j in range(n_f):
            if ris[j]:
                if cf>c[i][j]:
                    cf=c[i][j]
        cost+=cf
    return cost

df = pd.read_csv("es.csv")

nodes = 200
data = df[["city", "lat", "lng", "population"]].head(nodes).to_numpy()
city = np.reshape(df[["city"]].head(nodes).to_numpy(), nodes)
lat = np.reshape(df[["lat"]].head(nodes).to_numpy(), nodes)
lng = np.reshape(df[["lng"]].head(nodes).to_numpy(), nodes)
population = np.reshape(df[["population"]].head(nodes).to_numpy(), nodes).tolist()

n = data.shape[0]
n_f = n
n_c = n
distances = np.empty((n_c, n_f))
transport_cost = np.empty((n_c, n_f))
facility_cost = []

for i in range(n_c):
    for j in range(n_f):
        distances[i, j] = m.sqrt((lat[i]-lat[j])**2+(lng[i]-lng[j])**2)
        transport_cost[i, j] = distances[i, j]*population[i]/1000

c = transport_cost.tolist()

facility_cost = [1000]*n_f

f = facility_cost

pVal = 11 #number of facilities to activate

# plot nodes
positions=locate(f,c)

# simple plant location and plot solution
SPL_ris=SPL(f,c, population)

print('Cost Evaluation SPL:',evaluate_out(f,c,SPL_ris))
showsol(f,c,SPL_ris,positions)

# Constructive algorithm and plot solution
const_ris=constructive(f,c,pVal)


print('Cost Evaluation Constructive:',evaluate_out(f,c,const_ris))

showsol(f,c,const_ris,positions)